In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
df=pd.read_csv("Spam-Data.csv")

In [3]:
X = df['Message']
y = df['Category']

In [4]:
X.head()

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: Message, dtype: object

In [5]:
y.head()

0     ham
1     ham
2    spam
3     ham
4     ham
Name: Category, dtype: object

In [6]:
# Convert labels to numeric using LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y)

[0 0 1 ... 0 0 0]


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000)

# Fit the tokenizer on the text data
tokenizer.fit_on_texts(X_train)
tokenizer.fit_on_texts(X_test)

# Convert the text data into integer sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to a maximum length of 10
max_length = 10
padded_sequences = pad_sequences(X_train, maxlen=max_length, padding='post')
padded_sequences = pad_sequences(X_test, maxlen=max_length, padding='post')



In [10]:
# Convert the labels to numpy arrays
import numpy as np
y_train = np.array(y_train)
y_test = np.array(y_test)


In [11]:
X_train = tokenizer.sequences_to_matrix(X_train, mode='binary')
X_test = tokenizer.sequences_to_matrix(X_test, mode='binary')

In [12]:
# Convert the labels into one-hot encoded vectors
num_classes = np.max(y_train) + 1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [13]:
# Create the sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(10000,)))
model.add(Dense(num_classes, activation='softmax'))

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [15]:
# Train the model on the training data
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
126/126 [==============================] - 3s 17ms/step - loss: 0.2212 - accuracy: 0.9474 - val_loss: 0.1050 - val_accuracy: 0.9731
Epoch 2/10
126/126 [==============================] - 1s 12ms/step - loss: 0.0486 - accuracy: 0.9893 - val_loss: 0.0899 - val_accuracy: 0.9731
Epoch 3/10
126/126 [==============================] - 1s 10ms/step - loss: 0.0275 - accuracy: 0.9928 - val_loss: 0.0835 - val_accuracy: 0.9776
Epoch 4/10
126/126 [==============================] - 2s 14ms/step - loss: 0.0174 - accuracy: 0.9955 - val_loss: 0.0906 - val_accuracy: 0.9798
Epoch 5/10
126/126 [==============================] - 2s 14ms/step - loss: 0.0112 - accuracy: 0.9975 - val_loss: 0.1044 - val_accuracy: 0.9753
Epoch 6/10
126/126 [==============================] - 1s 11ms/step - loss: 0.0070 - accuracy: 0.9978 - val_loss: 0.1072 - val_accuracy: 0.9798
Epoch 7/10
126/126 [==============================] - 2s 12ms/step - loss: 0.0045 - accuracy: 0.9988 - val_loss: 0.1125 - val_accuracy: 0.9798

In [16]:
# Create the Autoencoder
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

input_layer = Input(shape=(X_train.shape[1],))

# Encoder
encoder_layer1 = Dense(512, activation='relu')(input_layer)
encoder_layer2 = Dense(256, activation='relu')(encoder_layer1)
encoder_layer3 = Dense(128, activation='relu')(encoder_layer2)
encoder_layer4 = Dense(64, activation='relu')(encoder_layer3)


In [17]:
# Latent space
latent_space = Dense(32, activation='relu')(encoder_layer4)

In [18]:
# Decoder
decoder_layer1 = Dense(64, activation='relu')(latent_space)
decoder_layer2 = Dense(128, activation='relu')(decoder_layer1)
decoder_layer3 = Dense(256, activation='relu')(decoder_layer2)
decoder_layer4 = Dense(512, activation='relu')(decoder_layer3)

In [19]:
# Output
output_layer = Dense(X_train.shape[1], activation='sigmoid')(decoder_layer4)

In [20]:

# Define the Autoencoder model
autoencoder = Model(inputs=input_layer, outputs=output_layer)

In [21]:
# Compile the Autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [22]:
# Fit the Autoencoder on the training data
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, shuffle=True)

Epoch 1/10
140/140 [==============================] - 27s 179ms/step - loss: 0.0873
Epoch 2/10
140/140 [==============================] - 24s 174ms/step - loss: 0.0081
Epoch 3/10
140/140 [==============================] - 26s 183ms/step - loss: 0.0078
Epoch 4/10
140/140 [==============================] - 24s 170ms/step - loss: 0.0078
Epoch 5/10
140/140 [==============================] - 23s 166ms/step - loss: 0.0078
Epoch 6/10
140/140 [==============================] - 25s 176ms/step - loss: 0.0078
Epoch 7/10
140/140 [==============================] - 23s 163ms/step - loss: 0.0078
Epoch 8/10
140/140 [==============================] - 26s 185ms/step - loss: 0.0078
Epoch 9/10
140/140 [==============================] - 23s 167ms/step - loss: 0.0078
Epoch 10/10
140/140 [==============================] - 24s 171ms/step - loss: 0.0078


In [23]:
# Generate the malicious data
num_poison_samples = int(len(X_train) * 0.1)
poisoned_data = X_train[:num_poison_samples]
malicious_data = autoencoder.predict(poisoned_data)

14/14 [==============================] - 1s 30ms/step


In [24]:
# Create the labels for the malicious data
malicious_labels = np.full((num_poison_samples,), num_classes - 1)

In [25]:
# Add the malicious data to the training set
train_data_malicious = np.concatenate((X_train, malicious_data), axis=0)
train_labels_malicious = np.concatenate((y_train, to_categorical(malicious_labels, num_classes)), axis=0)

In [26]:
# Retrain the model on the poisoned data
model.fit(train_data_malicious, train_labels_malicious, epochs=10, batch_size=32, validation_split=0.1)


Epoch 1/10
138/138 [==============================] - 2s 13ms/step - loss: 0.0129 - accuracy: 0.9973 - val_loss: 9.3416 - val_accuracy: 0.0916
Epoch 2/10
138/138 [==============================] - 1s 10ms/step - loss: 0.0083 - accuracy: 0.9982 - val_loss: 8.5370 - val_accuracy: 0.0937
Epoch 3/10
138/138 [==============================] - 1s 10ms/step - loss: 0.0053 - accuracy: 0.9986 - val_loss: 9.8637 - val_accuracy: 0.0916
Epoch 4/10
138/138 [==============================] - 1s 10ms/step - loss: 0.0041 - accuracy: 0.9989 - val_loss: 9.5707 - val_accuracy: 0.0916
Epoch 5/10
138/138 [==============================] - 1s 10ms/step - loss: 0.0028 - accuracy: 0.9989 - val_loss: 9.5170 - val_accuracy: 0.0916
Epoch 6/10
138/138 [==============================] - 2s 11ms/step - loss: 0.0021 - accuracy: 0.9993 - val_loss: 10.7520 - val_accuracy: 0.0916
Epoch 7/10
138/138 [==============================] - 2s 15ms/step - loss: 0.0016 - accuracy: 0.9993 - val_loss: 10.8823 - val_accuracy: 0.09

In [27]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

35/35 [==============================] - 0s 4ms/step - loss: 0.0705 - accuracy: 0.9919
Test accuracy: 0.9919282793998718
